
### 데이터셋 준비 

In [ ]:
import tensorflow as tf
tf.random.set_seed(777)

from tensorflow.keras.datasets import mnist

# 테스트셋은 사용하지 않기 때문에 생략합니다.
(x_train, y_train), _ = mnist.load_data()
x_train = x_train / 255.0

# 채널을 추가합니다.(28, 28) -> (28, 28, 1)
x_train = x_train[..., tf.newaxis]

from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, 
                                                  test_size = 0.3, 
                                                  random_state = 777)

print(f'x_train shape: {x_train.shape} \nx_val shape: {x_val.shape}')

### Dataset 정의

In [ ]:
# 학습 데이터셋 객체를 생성합니다.
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.shuffle(1000).batch(32)
train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)

# 검증 데이터셋 객체를 생성합니다.
val_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(32)

### 모델 정의

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Input
from tensorflow.keras import Model

def get_model():
    inputs = Input(shape = (28, 28, 1))

    x = Conv2D(32, 3, activation = 'relu')(inputs)
    x = Flatten()(x)
    x = Dense(128, activation = 'relu')(x)
    x = Dense(10, activation = 'softmax')(x)

    model = Model(inputs = inputs, outputs = x)

    return model

model = get_model()

### loss 및 optimizer 정의

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

learning_rate = 0.0001

loss_object = SparseCategoricalCrossentropy()
optimizer = Adam(learning_rate = learning_rate, beta_1 = 0.9, beta_2 = 0.98, epsilon = 1e-9)

### Metric 정의

In [ ]:
from tensorflow.keras.metrics import Mean
from tensorflow.keras.metrics import SparseCategoricalAccuracy

train_loss = Mean(name='train_loss')
train_accuracy = SparseCategoricalAccuracy(name='train_accuracy')

val_loss = Mean(name='val_loss')
val_accuracy = SparseCategoricalAccuracy(name='val_accuracy')

### 학습 과정 정의

In [ ]:
# 학습 스텝을 정의합니다.
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        outputs = model(images, training=True)
        # 배치 학습 데이터의 개별 손실값을 구합니다.
        loss = loss_object(labels, outputs)

    # 손실값 참고하여, 그래디언트를 구합니다.
    gradients = tape.gradient(loss, model.trainable_variables)
    # 모델의 가중치를 업데이트합니다.
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # 학습 손실값을 계산합니다.
    train_loss(loss)
    # 학습 평가지표를 계산합니다.
    train_accuracy(labels, outputs)

# 검증 스텝을 정의합니다.
@tf.function
def val_step(images, labels):
    outputs = model(images, training=False)
    # 배치 검증 데이터의 개별 손실값을 구합니다.
    v_loss = loss_object(labels, outputs)

    # 검증 손실값을 계산합니다.
    val_loss(v_loss)
    # 검증 평가지표를 계산합니다.
    val_accuracy(labels, outputs)

### 학습 진행

In [ ]:
EPOCHS = 2

# 수준: epoch
for epoch in range(EPOCHS):
    # 다음 에폭을 위해 지표를 초기화합니다.
    train_loss.reset_states()
    train_accuracy.reset_states()
    val_loss.reset_states()
    val_accuracy.reset_states()

    # 수준: step
    for images, labels in train_ds:
        train_step(images, labels)

    # 수준: step
    for val_images, val_labels in val_ds:
        val_step(val_images, val_labels)

    print('Epoch: {}, train_loss: {}, train_acc: {} val_loss: {}, val_acc: {}'.format(
          epoch + 1,
          train_loss.result(), train_accuracy.result() * 100,
          val_loss.result(), val_accuracy.result() * 100))

<hr></hr>

# TQDM 및 training setup 추가


### 데이터셋 준비 

In [ ]:
from tqdm import tqdm
import gc

import tensorflow as tf
tf.random.set_seed(777)

from tensorflow.keras.datasets import mnist

# 테스트셋은 사용하지 않기 때문에 생략합니다.
(x_train, y_train), _ = mnist.load_data()
x_train = x_train / 255.0

# 채널을 추가합니다.(28, 28) -> (28, 28, 1)
x_train = x_train[..., tf.newaxis]

from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, 
                                                  test_size = 0.3, 
                                                  random_state = 777)

print(f'x_train shape: {x_train.shape} \nx_val shape: {x_val.shape}')

### Dataset 정의

In [ ]:
# 학습 데이터셋 객체를 생성합니다.
train_ds = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_ds = train_ds.shuffle(1000).batch(32)
train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)

# 검증 데이터셋 객체를 생성합니다.
val_ds = tf.data.Dataset.from_tensor_slices((x_val, y_val)).batch(32)

### 모델 정의

In [ ]:
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Input
from tensorflow.keras import Model

def get_model():
    inputs = Input(shape = (28, 28, 1))

    x = Conv2D(32, 3, activation = 'relu')(inputs)
    x = Flatten()(x)
    x = Dense(128, activation = 'relu')(x)
    x = Dense(10, activation = 'softmax')(x)

    model = Model(inputs = inputs, outputs = x)

    return model

model = get_model()

### loss 및 optimizer 정의

In [ ]:
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam

learning_rate = 0.0001

loss_object = SparseCategoricalCrossentropy()
optimizer = Adam(learning_rate = learning_rate, beta_1 = 0.9, beta_2 = 0.98, epsilon = 1e-9)

### Metric 정의

In [ ]:
from tensorflow.keras.metrics import Mean
from tensorflow.keras.metrics import SparseCategoricalAccuracy

train_loss = Mean(name='train_loss')
train_accuracy = SparseCategoricalAccuracy(name='train_accuracy')
train_accuracy_mean = Mean(name = 'train_accuracy_mean')

val_loss = Mean(name='val_loss')
val_accuracy = SparseCategoricalAccuracy(name='val_accuracy')
val_accuracy_mean = Mean(name = 'val_accuracy_mean')

### 학습 과정 정의

In [ ]:
# 학습 스텝을 정의합니다.
@tf.function
def train_step(images, labels):
    with tf.GradientTape() as tape:
        outputs = model(images, training=True)
        # 배치 학습 데이터의 개별 손실값을 구합니다.
        loss = loss_object(labels, outputs)
    
    # 손실값 참고하여, 그래디언트를 구합니다.
    gradients = tape.gradient(loss, model.trainable_variables)
    # 모델의 가중치를 업데이트합니다.
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    # 학습 손실값을 계산합니다.
    train_loss(loss)
    # 학습 평가지표를 계산합니다.
    train_accuracy(labels, outputs)
    train_accuracy_mean(train_accuracy.result())
    
    return loss

# 검증 스텝을 정의합니다.
@tf.function
def val_step(images, labels):
    outputs = model(images, training=False)
    # 배치 검증 데이터의 개별 손실값을 구합니다.
    v_loss = loss_object(labels, outputs)

    # 검증 손실값을 계산합니다.
    val_loss(v_loss)
    # 검증 평가지표를 계산합니다.
    val_accuracy(labels, outputs)
    val_accuracy_mean(val_accuracy.result())
    
    return v_loss

### 학습 진행

In [ ]:
loss_plot, val_loss_plot = [], []

EPOCHS = 2

# 수준: epoch
for epoch in range(EPOCHS):
    total_loss, total_val_loss = 0, 0
    
    # 다음 에폭을 위해 지표를 초기화합니다.
    train_loss.reset_states()
    train_accuracy.reset_states()
    train_accuracy_mean.reset_states()
    
    val_loss.reset_states()
    val_accuracy.reset_states()
    val_accuracy_mean.reset_states()
    
    tqdm_dataset = tqdm(enumerate(train_ds))
    
    # Train Step, 수준: step
    for batch, (images, labels) in tqdm_dataset:
        batch_loss = train_step(images, labels)
        total_loss += batch_loss
        
        tqdm_dataset.set_postfix({
            'Epoch': epoch + 1,
            'Loss': '{:06f}'.format(batch_loss.numpy()),
            'Total Loss': '{:06f}'.format(total_loss / (batch + 1)),
            'Accracy': train_accuracy_mean.result().numpy()
        })
        
        if batch % 30 == 0:
            gc.collect()
        
    loss_plot.append(total_loss / (batch + 1))
    
    tqdm_dataset_val = tqdm(enumerate(val_ds))
    
    # Validation Step, 수준: step
    for batch, (val_images, val_labels) in tqdm_dataset_val:
        batch_val_loss = val_step(val_images, val_labels)
        total_val_loss += batch_val_loss
        
        tqdm_dataset_val.set_postfix({
            'Epoch': epoch + 1,
            'Val Loss': '{:06f}'.format(batch_val_loss.numpy()),
            'Val Total Loss': '{:06f}'.format(total_val_loss / (batch + 1)),
            'Val Accracy': val_accuracy_mean.result().numpy()
        })
        
    val_loss_plot.append(total_val_loss / (batch + 1))

In [ ]:
plt.plot(loss_plot, label = 'train_loss')
plt.plot(val_loss_plot, label = 'val_loss')
plt.legend()
plt.show()